In [1]:
# import packages
import pandas as pd
import time
import datetime
import numpy as np
import matplotlib.pyplot as plt
import chromedriver_binary
import random
from datetime import timedelta, date
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.options import Options
from selenium.common.exceptions import NoSuchElementException

In [2]:
#parametros e site
params = {'login':'04222526380',
          'password':'339715'}

driver = webdriver.Edge() # escolhendo o driver
driver.get("https://radar.serpro.gov.br/")# select site


In [3]:
def login():
  currentUrl = driver.current_url;
  if currentUrl == "https://radar.serpro.gov.br/main.html#/login":
    try:
      element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'usuario')))       
      element.send_keys(params['login'])
      element = driver.find_element(By.ID, "password")
      element.send_keys(params['password'])
      element = driver.find_element_by_id('Entrar')
      element.click()
      print("login feito")
      #acess the search of the taxs
      element = WebDriverWait(driver, 12).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 
      'body > div.bg > div > div.bg.ng-scope > div.ng-scope > header > nav > div > ul > li.menu-principal.menu-dropdown')))
      element.click()
      element = WebDriverWait(driver, 12).until(EC.presence_of_element_located((By.CSS_SELECTOR, 
      'body > div > div > div.bg.ng-scope > div.ng-scope > header > nav > div > ul > li.menu-principal.menu-dropdown.open > ul > div > li:nth-child(1) > ul > li:nth-child(1) > a')))
      element.click()
      element = WebDriverWait(driver, 12).until(EC.element_to_be_clickable((By.ID,'abaBuscaAvancada')))
      element.click()
      print("Acesso Menu realizado")
    except Exception as e:
      print("Não foi possível fazer login")
      print(e)
  print("Operação completa")
  
login() 

C:\Users\Filipe\AppData\Local\Temp\ipykernel_10076\3153827461.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id('Entrar')


login feito
Acesso Menu realizado
Operação completa


In [ ]:
df_april = pd.read_excel("ResultadoConsultaInfracoes_abril.xlsx", skiprows=5)
df_april.columns = df_april.iloc[0]
df_april = df_april[1:]
df_test = df_april['N. Auto'].head(100)
for i in df_test:
    print(str(i))   

In [23]:
sne_list = []
site_list = []
date_hour_list = []
date_list = []
hour_list = []
code_list = []
reg_list = []

def reg_():
    reg = [reg.split(': ', 1)[1] for reg in reg_list]
    return(reg)

def site_():
    site = [site.split(': ', 1)[1] for site in site_list]
    return(site)

def code_():
    codes = [code.replace(' - ', '') for code in code_list]
    code = [code.split(': ', 1)[1] for code in codes]
    return(code)

def date_():
    date_hour = [date_hour.split(': ', 1)[1] for date_hour in date_hour_list]
    date = [date.split(' ', 1)[0] for date in date_hour]
    return(date)

def hour_():
    date_hour = [date_hour.split(': ', 1)[1] for date_hour in date_hour_list]
    hour = [hour.split(' ', 1)[1] for hour in date_hour]
    return(hour)

def souce_finn(auto):
    try:
        print("Inicio Pesquisa")
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'codigo')))
        element.clear()
        element.send_keys(str(auto))
        element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'btnBuscaAvancada')))
        element.click()
        element = WebDriverWait(driver, 12).until(EC.invisibility_of_element((By.CLASS_NAME, 'loading-root')))
        table_id = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#vm\.id > tbody')))
        print("Informação SEI:")
        rows = table_id.find_elements(By.TAG_NAME, "tr") 
        for row in rows:
            col = row.find_elements(By.TAG_NAME, "td")[11]
            print(col.text)
            sne_list.append(col.text)
        print("Acessando a caixa de informações")
        element = WebDriverWait(driver, 12).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#vm\.id > tbody > tr > td:nth-child(2) > a')))
        element.click()
        element = WebDriverWait(driver, 12).until(EC.invisibility_of_element((By.CLASS_NAME, 'loading-root')))
        
         # get site
        site = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[5]/div/div[1]')[0].text
        print(site)
        site_list.append(site) 
        
        # get code 
        code = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[6]/div/div[1]')[0].text
        print(code)
        code_list.append(code)

         # get reg 
        element = WebDriverWait(driver, 12).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#imprimirMulta > fieldset:nth-child(9) > div > div:nth-child(3) > span")))
        if element.is_displayed():
            reg = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[7]/div/div[7]')[0].text
        else:
            reg = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[6]/div/div[7]')[0].text
        print(reg)
        reg_list.append(reg)

        # get  date and hours
        date_hour = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[5]/div/div[5]')[0].text
        print(date_hour)
        date_hour_list.append(date_hour)
               
        driver.find_element_by_css_selector("#btnFecharDetalhe").click()
        
    except Exception as e:
      print("Acesso Pesquisa")
      print(e)
      print("Operação completa")


# "//*[@id="imprimirMulta"]/fieldset[7]/div/div[7]"



In [ ]:
for i in df_test:
    print("Auto:", i)
    souce_finn(i)

reg_list = reg_()
site_list = site_()
code_list = code_()
date_list = date_()
hour_list = hour_()
reg_list = reg_()